In [4]:
#response_excellentoriented = requests.get(excellentoriented_url)
#response_excellentunoriented = requests.get(excellentunoriented_url)
#response_fairoriented = requests.get(fairoriented_url)
#response_fairunoriented = requests.get(fairunoriented_url)

import sqlite3
import requests
import zipfile
import io
import re
import json

def extract_wavelength(filename):
    # Extract the wavelength using regular expression
    wavelength_match = re.search(r'Raman__([\d.]+)__', filename)
    if wavelength_match:
        return wavelength_match.group(1)
    else:
        return None

def extract_file_number(filename):
    # Extract the file number using regular expression
    file_number_match = re.search(r'__(\d+)\.txt$', filename)
    if file_number_match:
        return file_number_match.group(1)
    else:
        return None

def extract_elements(chemical_formula):
    # Extract elements from chemical formula using regular expression
    element_symbols = re.findall(r'[A-Z][a-z]*', chemical_formula)
    return ', '.join(element_symbols)

conn = sqlite3.connect('RRUFFRaman_databaseSEARCH.db')  # Replace 'your_database.db' with your database filename
cursor = conn.cursor()
create_table_query = '''
    CREATE TABLE IF NOT EXISTS database_tableSEARCH (
        id INTEGER PRIMARY KEY,
        filename TEXT,
        x_data REAL,
        y_data REAL
    )
'''
cursor.execute(create_table_query)
conn.commit()

excellentoriented_url = "https://rruff.info/zipped_data_files/raman/excellent_oriented.zip"
excellentunoriented_url = "https://rruff.info/zipped_data_files/raman/excellent_oriented.zip"
fairoriented_url = "https://rruff.info/zipped_data_files/raman/fair_oriented.zip"
fairunoriented_url = "https://rruff.info/zipped_data_files/raman/fair_oriented.zip"

response_excellentoriented = requests.get(excellentoriented_url)
response_excellentunoriented = requests.get(excellentunoriented_url)
response_fairoriented = requests.get(fairoriented_url)
response_fairunoriented = requests.get(fairunoriented_url)

if (response_excellentoriented.status_code == 200) and \
   (response_excellentunoriented.status_code == 200) and \
   (response_fairoriented.status_code == 200) and \
   (response_fairunoriented.status_code == 200):
    with io.BytesIO(response_excellentoriented.content) as zip_stream:
        with zipfile.ZipFile(zip_stream) as zip_ref:
            for filename in zip_ref.namelist():
                with zip_ref.open(filename) as file:
                    content = file.read().decode('utf-8')
                    lines = content.split('\n')
                    
                    # Outside the loop that processes files
                    data_to_insert = []

                    # Inside the loop that processes lines
                    for line in lines:
                        if line.startswith("##") or line.strip() == "":
                            continue  # Skip metadata and empty lines
                        x, y = map(float, line.split(', '))
                        data_to_insert.append((x, y))

                    # After processing a file
                    if data_to_insert:
                        cursor.execute("INSERT INTO database_tableSEARCH (filename, x_data, y_data) VALUES (?, ?, ?)",
                                       (filename, json.dumps([x for x, _ in data_to_insert]), json.dumps([y for _, y in data_to_insert])))
                        data_to_insert = []  # Reset the data list for the next file


    conn.commit()
    print("Data download and insertion completed.")
else:
    print("Failed to download data.")

conn.close()

Data download and insertion completed.
